# Prediction of overlapping spans with spaCy's SpanCategorizer

**Motivation**:

Annotations in GGPONC are often overlapping or nested.

For instance, `Versagen einer Behandlung mit Oxaliplatin und Irinotecan`
- is a *Finding*
- which contains a *Therapeutic Procedure*: `Behandlung mit Oxaliplatin und Irinotecan`:
    - which in turn contains two *Clinical Drug* names: (`Oxaliplatin` and `Irinotecan`).

Standard IOB-encoded labels, and most NER implementations, can only model one label per token, so by default we consider the longest surrounding mention span only in the IOB-based / HuggingFace implementation (in this case, the *Finding*).

**Solution**:

Instead of token-level labels, we use spaCy's new [SpanCategorizer](https://spacy.io/api/spancategorizer/) implementation to predict overlapping mention spans as a SpanGroup in a spaCy document.

## Training

See the `spacy` folder in the root directory of the project. The model configuration can be found at `configs` and training can be run through a spaCy project (see `spacy/run_training.sh`). 

*Note:* We have currently not optimized the many hyperparameters related to span suggestion and model training. However, performance is close to the HuggingFace models evaluated on non-nested mention spans.

## Inference

In [12]:
import sys
sys.path.append('../spacy')

In [15]:
import spacy
from spacy.tokens import Doc, Span
import snomed_spans #TODO: import needed to enable custom spaCy components, is there another way?

In [16]:
nlp = spacy.load('../data/models/spacy')

### Grascco Samples

In [21]:
doc = nlp("""6.04.2029: Nachdebridement am Kopf, VAG-Wechsel linke Hand""")


In [20]:
doc = nlp("6.04.2029: Nachdebridement am Kopf, VAG-Wechsel linke Hand""Röntgen : Rippstein I : Gute Hüftkopfepiphysenkonturgebung , minimale Lateralisation , li. etwas stärker als re. , noch übergreifende Pfannendächer , Shenton-Menard-Linie nicht wesentlich unterbrochen , Pfannendachwinkel Ii. 30° , re. ebenfalls knapp 30° .""")


In [ ]:
for s in sorted(list(doc.spans['snomed']), key=lambda s: s.start):
    print(s, s.label_)

## Initial Sentence Based Processing + Eval

In [4]:
import json

nlp = spacy.load('../data/models/spacy')

def find_annotated_entities(sentence, document):
    s_offsets = sentence['offsets']
    entity_list = []
    for entity in document['entities']:
        if entity is None:
            break
        elif(entity['offsets'][0][0] > s_offsets[0][1]): 
            break
        elif (entity['offsets'][0][0] >=  s_offsets[0][0] and entity['offsets'][0][1] <= s_offsets[0][1]):
            entity_list.append(entity)
    return entity_list

tp = 0
e_count = 0
t_count = 0

def compare_findings(predi_entities, truth_entities, sentence):
    sentence_delta = sentence['offsets'][0][0]
    global tp, e_count, t_count
    for e in list(predi_entities.spans['snomed']):
        e_count+=1
        for t in truth_entities:
            if e.label_ == t['type'] and e.start_char == t['offsets'][0][0]-sentence_delta and e.end_char == t['offsets'][0][1]-sentence_delta:
                tp+=1
                break
    t_count += len(truth_entities)


json_path = "/Users/leon.sarodnik/Documents/GitHub/ggponc_annotation/GraSSco/grassco_anno_2023-01-05_0021/json/fine/long/test.json"
json_path_new = "/Users/leon.sarodnik/Documents/GitHub/ggponc_annotation/GraSSco/grascco_hpi_anno_2023_02_08/annotations/json/fine/long/all_short.json"

with open(json_path_new) as json_file:
    data = json.load(json_file)


for document in data:
    print("Current document_id: "+document['document_id'])
    for sentence in document['passages']:
        nlp_findings = nlp(sentence['text'], disable=["tok2vec", "tagger", "attribute_ruler","lemmatizer"]) #disable_components (standard NER tagger etc.) - "parser" is helpful (10 matches less when not used)
        manual_findings = find_annotated_entities(sentence, document)
        compare_findings(nlp_findings, manual_findings, sentence)



KeyboardInterrupt



## Processing Pipeline

In [1]:
import json

# Extract paths to configuration files
CONFIG = {
    'json_path':'../data/GraSSco/grassco_anno_2023-01-05_0021/json/fine/long/test.json',
    'json_path_new': '../data/GraSSco/grascco_hpi_anno_2023_02_08/annotations/json/fine/long/all_short.json',
    'ggponc_full_set': '../data/annotations/json/fine/long/all.json',
    'train_test_split': '../data/annotations/splits.csv'
}


### Get GGPONC Test Data set as given in splits.csv

In [120]:
import csv
import json
import os


# Open the CSV file and read its contents into a dictionary
with open(CONFIG['train_test_split'], 'r') as csvfile:
    next(csvfile)
    csvdata = {row[0]: row[1] for row in csv.reader(csvfile)}

test_ids = set()
# Get the set of test file IDs
for file_name, split_value in csvdata.items():
    if split_value == 'test':
        test_ids.add(os.path.splitext(file_name)[0])


# Load the JSON data from the file and filter it for the test split
with open(CONFIG['ggponc_full_set'], 'r') as jsonfile:
    jsondata = json.load(jsonfile)
    test_data = [doc for doc in jsondata if os.path.splitext(doc['document_id'])[0] in test_ids]
        

## Sentence Based NER processing and saving result as BigBio (json) (predicted entities only)

In [ ]:
with open(CONFIG['json_path']) as json_file:
    data = json.load(json_file)

predicted_findings = []

for document in data:
    print("Current document_id: " + document['document_id'])
    entities = []
    # use pipe, disable_components in pipe (standard NER tagger etc.) - "parser" is helpful (10 matches less when not used)
    for i, doc in enumerate(nlp.pipe([passage['text'] for passage in document['passages'] if 'text' in passage], disable=["tok2vec", "tagger", "attribute_ruler","lemmatizer"])):
        entity = []
        sentence_offset = document['passages'][i]['offsets'][0][0]
        for ent in doc.spans['snomed']:
            entity.append({
            "text": [ent.text],
            "type": ent.label_,
            "offsets": [[ent.start_char+sentence_offset,ent.end_char+sentence_offset]]
        })

        entities.extend(entity)
        

    output_dict = {
        "document_id": document['document_id'],
        "entities": entities
    }
    
    predicted_findings.append(output_dict)
    
# Write the output to json file
import os
output_path = os.path.splitext(json_file.name)[0] + '_NER_processed.json'

with open(output_path, "w") as outfile:
    json.dump(predicted_findings, outfile,ensure_ascii=False)

## Error Analysis

In [2]:
#load predicted json data
with open('../data/GraSSco/grassco_anno_2023-01-05_0021/json/fine/long/test_NER_processed.json') as json_file:
    predicted_findings = json.load(json_file)

### Traditional Error Analysis (F1 Score based on TP, FN, FP)

In [117]:
from spacy import util

def compare_findings(pred_entities, truth_entities):
    true_positive_count = sum(1 for e in pred_entities if any(e['type'] == t['type'] and
                                     e['offsets'] == t['offsets']
                                     for t in truth_entities))
    return true_positive_count, len(pred_entities), len(truth_entities)


with open(CONFIG['json_path']) as json_file:
    data_t = json.load(json_file)

tp = e_count = t_count = 0

for i_doc, doc_p in enumerate(predicted_findings):
        tp_i, e_count_i, t_count_i, = compare_findings(doc_p['entities'], data[i_doc]['entities'])
        tp += tp_i
        e_count += e_count_i
        t_count += t_count_i

In [118]:
fn = t_count-tp
fp = e_count-tp

print(f'Actual Entities: {t_count}')
print(f'Predicted Entities: {e_count}')

print(f'True Positives: {tp}')
print(f'False Positives: {fp}')
print(f'False Negatives: {fn}')

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1_score:.2f}')


#result for fine/long

#Actual Entities: 6194
#Predicted Entities: 5171
#True Positives: 2584
#False Positives: 2587
#False Negatives: 3610
#Precision: 0.50
#Recall: 0.42
#F1 Score: 0.45

Actual Entities: 6194
Predicted Entities: 5171
True Positives: 2584
False Positives: 2587
False Negatives: 3610
Precision: 0.50
Recall: 0.42
F1 Score: 0.45


### NER context related Error Analysis (based on FairEval)

In [18]:
import sys
sys.path.append('../experiments')

from error_analysis import _ner_error_analyis

def compare_findings(pred_entities, truth_entities):
    pred_spans = []
    gt_spans = []
    
    for p in pred_entities:
        pred_spans.append({
            'start': p['offsets'][0][0],
            'end': p['offsets'][0][1],
            'entity_group': p['type'],
            'word': p['text'][0]
        })

    for t in truth_entities:
        gt_spans.append({
            'start': t['offsets'][0][0],
            'end': t['offsets'][0][1],
            'entity_group': t['type'],
            'word': t['text'][0]
        })
    
    return _ner_error_analyis(pred_spans, gt_spans)


with open(CONFIG['json_path']) as json_file:
    data = json.load(json_file)

errors = []

for i_doc, doc_p in enumerate(predicted_findings):
        errors_i = compare_findings(doc_p['entities'], data[i_doc]['entities'])
        errors.extend(errors_i)


In [19]:
# get stats and export errors as csv
import pandas as pd
stats_df = pd.DataFrame(errors)

error_count = stats_df.groupby('category').size()
stats_df.to_csv('error_analysis.csv')
error_count

category
boundary_error_larger          31
boundary_error_overlap         14
boundary_error_smaller        362
boundary_error_span_count     171
false_negative               2853
false_positive                952
label_boundary_error          774
labeling_error                337
true_positive                1590
dtype: int64

## Document Based Processing

In [ ]:
import json

#nlp = spacy.load('../data/models/spacy')

def get_string_sentences(docs):
    return ['\n'.join([sentence['text'] for sentence in doc['passages']]) for doc in docs]

def find_annotated_entities(annotated_doc):
    return [entity for entity in annotated_doc['entities'] if entity]

def compare_findings(predicted_findings, truth_entities):
    predicted_findings = sorted(predicted_findings, key=lambda x: x.start_char)
    truth_set = set(t['offsets'][0][0] for t in truth_entities)
    tp, e_count, t_count = 0, len(predicted_findings), len(truth_entities)
    for e in predicted_findings:
        if e.start_char in truth_set:
            t = next(t for t in truth_entities if t['offsets'][0][0] == e.start_char)
            if e.label_ == t['type'] and e.start_char == t['offsets'][0][0] and e.end_char == t['offsets'][0][1]:
                tp += 1
    return tp, e_count, t_count

tp, e_count, t_count = 0, 0, 0

json_path = "../data/GraSSco/grassco_anno_2023-01-05_0021/json/fine/long/test.json"
json_path_new = "../data/GraSSco/grascco_hpi_anno_2023_02_08/annotations/json/fine/long/all_short.json"
ggponc_test_set = "../data/annotations/json/fine/long/all.json"

with open(ggponc_test_set) as json_file:
    data = json.load(json_file)

print("Processing all documents... this takes up to multiple minutes hang tight!")

# use pipe, disable_components in pipe (standard NER tagger etc.) - "parser" is helpful (10 matches less when not used)
for i, doc in enumerate(nlp.pipe(get_string_sentences(data), disable=["tok2vec", "attribute_ruler", "lemmatizer"])):
    predicted_findings = [(ent) for ent in list(doc.spans['snomed'])]
    manual_findings = find_annotated_entities(data[i])
    tp_i, e_count_i, t_count_i = compare_findings(predicted_findings, manual_findings)
    tp += tp_i
    e_count += e_count_i
    t_count += t_count_i


In [32]:
fn = t_count-tp
fp = e_count-tp

print(f'Actual Entities: {t_count}')
print(f'Predicted Entities: {e_count}')

print(f'True Positives: {tp}')
print(f'False Positives: {fp}')
print(f'False Negatives: {fn}')

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1_score:.2f}')


#sentence based result for all-short

#Actual Entities: 195
#Predicted Entities: 187
#True Positives: 87
#False Positives: 100
#False Negatives: 108
#Precision: 0.47
#Recall: 0.45
#F1 Score: 0.46

#document based result for all-short

#Actual Entities: 195
#Predicted Entities: 177
#True Positives: 81
#False Positives: 96
#False Negatives: 114
#Precision: 0.46
#Recall: 0.42
#F1 Score: 0.44

Actual Entities: 195
Predicted Entities: 177
True Positives: 81
False Positives: 96
False Negatives: 114
Precision: 0.46
Recall: 0.42
F1 Score: 0.44


## Spacy Native Eval Approach (in progress)

In [ ]:

doc_bin = DocBin().from_disk(manual_annotated_file)


gold_docs  = list(doc_bin.get_docs(nlp.vocab))

#scorer = Scorer()
#scores = scorer.score(examples)

#gold_annotation = spacy.Corpus.v1(manual_annotated_file, gold_preproc=True)
docs = []
for file in files:
    docs.append(file.read_text(encoding="utf-8"))
print("Files merged...")


# Loop over the gold standard data
#for gold_doc in gold_docs:
#    ents1 = [(gold_doc.text, gold_doc.label_) for ent in gold_doc.ents]
    # Process the text with the model
# Compare the model's predicted annotations with the gold standard
#for doc in docs:
#    ents2 = [(doc.text, doc.label_) for ent in doc.ents]

print("NLP Pipe...")
docs_all = nlp.pipe(gold_docs, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"],batch_size = 10)


print(gold_docs[1])
print("Building Examples...")
examples = []
for  i, doc in enumerate(docs_all):
    examples.append(Example((doc), gold_docs[i]))
    


scorer = Scorer(nlp)

print("eval...")
scores = scorer.score(examples)


#example_object = iter(docs)


#examples = Example(Doc.from_docs(docs), Doc.from_docs(gold_docs))

scorer.score()


#print("Entities F-Score:", scorer.scores["ents_f"])
#print("Entities Precision:", scorer.scores["ents_p"])
#print("Entities Recall:", scorer.scores["ents_r"])


### Visualizing NER
Spacy has a library called “displaCy” which helps us to explore the behaviour of the entity recognition model interactively.

If you are training a model, it’s very useful to run the visualization yourself.

You can pass a Doc or a list of Doc objects to displaCy and run displacy.serve to run the webserver, or displacy.render to generate the raw mark-up.


In [ ]:

#Import the displaCy library
from spacy import displacy
